<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fbigdatamagazine.es%2Fwp-content%2Fuploads%2F2023%2F02%2FFOTO-OK-BDM-DIG-DATA-MAGAZINE.jpg&f=1&nofb=1&ipt=4061921fa0da07483f83edb036d31f25545b2cae889c7eeefebd576f6e0fe5f4" style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

En este tutorial, aprenderás cómo construir y optimizar modelos con **gradient boosting**. Este método domina muchas competiciones de Kaggle y alcanza resultados de última generación en una variedad de conjuntos de datos.

# Introducción

Durante gran parte de este curso, has hecho predicciones con el método de bosques aleatorios, el cual logra un mejor rendimiento que un solo árbol de decisión simplemente promediando las predicciones de muchos árboles de decisión.

Nos referimos al método del bosque aleatorio como un "método de ensamblado". Por definición, los **métodos de ensamblado** combinan las predicciones de varios modelos (por ejemplo, varios árboles en el caso de los bosques aleatorios).

A continuación, aprenderemos sobre otro método de ensamblado llamado gradient boosting.

# Gradient Boosting

El **gradient boosting** es un método que realiza ciclos para añadir modelos de manera iterativa a un ensamblado.

Comienza inicializando el ensamblado con un solo modelo, cuyas predicciones pueden ser bastante ingenuas. (Incluso si sus predicciones son muy inexactas, las adiciones posteriores al ensamblado corregirán esos errores).

Luego, iniciamos el ciclo:
- Primero, usamos el ensamblado actual para generar predicciones para cada observación en el conjunto de datos. Para hacer una predicción, sumamos las predicciones de todos los modelos del ensamblado.
- Estas predicciones se usan para calcular una función de pérdida (como el [error cuadrático medio](https://es.wikipedia.org/wiki/Error_cuadr%C3%A1tico_medio), por ejemplo).
- Luego, usamos la función de pérdida para ajustar un nuevo modelo que será añadido al ensamblado. Específicamente, determinamos los parámetros del modelo de forma que al añadir este nuevo modelo al ensamblado, se reduzca la pérdida. (*Nota: El "gradiente" en "gradient boosting" se refiere al hecho de que se utiliza [descenso por gradiente](https://es.wikipedia.org/wiki/Descenso_del_gradiente) sobre la función de pérdida para determinar los parámetros de este nuevo modelo.*)
- Finalmente, añadimos el nuevo modelo al ensamblado, y...
- ... ¡repetimos!

![tut6_boosting](https://storage.googleapis.com/kaggle-media/learn/images/MvCGENh.png)

# Ejemplo

Comenzamos cargando los datos de entrenamiento y validación en `X_train`, `X_valid`, `y_train` y `y_valid`.



In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Leer los datos
data = pd.read_csv('/content/melb_data.csv')

# Seleccionar un subconjunto de predictores
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Seleccionar la variable objetivo
y = data.Price

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(X, y)


En este ejemplo, trabajarás con la librería XGBoost. **XGBoost** significa **extreme gradient boosting** (refuerzo de gradiente extremo), que es una implementación del gradient boosting con varias características adicionales enfocadas en el rendimiento y la velocidad. (_Scikit-learn tiene otra versión de gradient boosting, pero XGBoost presenta algunas ventajas técnicas._)

En la siguiente celda de código, importamos la API de scikit-learn para XGBoost ([`xgboost.XGBRegressor`](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)). Esto nos permite construir y ajustar un modelo tal como lo haríamos con scikit-learn. Como verás en la salida, la clase `XGBRegressor` tiene muchos parámetros ajustables — ¡pronto aprenderás sobre ellos!


In [3]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

También hacemos predicciones y evaluamos el modelo

In [4]:
from sklearn.metrics import mean_absolute_error

# Hacer predicciones con el conjunto de validación
predictions = my_model.predict(X_valid)

# Imprimir el error absoluto medio
print("Error Absoluto Medio: " + str(mean_absolute_error(predictions, y_valid)))

Error Absoluto Medio: 236856.0188351436


# Ajuste de Parámetros

XGBoost tiene varios parámetros que pueden afectar drásticamente la precisión y la velocidad de entrenamiento. Los primeros parámetros que deberías entender son:

### `n_estimators`
`n_estimators` especifica cuántas veces se ejecuta el ciclo de modelado descrito anteriormente. Es igual al número de modelos que incluimos en el ensamblado.

- Un valor demasiado _bajo_ provoca _subajuste_ (_underfitting_), lo que conduce a predicciones inexactas tanto en los datos de entrenamiento como en los de prueba.
- Un valor demasiado _alto_ provoca _sobreajuste_ (_overfitting_), lo que produce predicciones precisas en los datos de entrenamiento, pero inexactas en los datos de prueba (_que son los que realmente nos importan_).

Los valores típicos oscilan entre 100 y 1000, aunque esto depende mucho del parámetro `learning_rate`, que se explicará a continuación.

Aquí tienes el código para establecer el número de modelos en el ensamblado:



In [39]:
# Crear el modelo con 500 árboles (estimadores) en el ensamblado
my_model = XGBRegressor(n_estimators=500)

# Entrenar el modelo con los datos de entrenamiento
my_model.fit(X_train, y_train,
             eval_set=[(X_valid, y_valid)],
             verbose=True)


[0]	validation_0-rmse:535046.34013
[1]	validation_0-rmse:477517.68097
[2]	validation_0-rmse:445198.04336
[3]	validation_0-rmse:425102.65133
[4]	validation_0-rmse:411945.25967
[5]	validation_0-rmse:405890.44880
[6]	validation_0-rmse:400547.88224
[7]	validation_0-rmse:396933.52512
[8]	validation_0-rmse:393692.87844
[9]	validation_0-rmse:388803.91233
[10]	validation_0-rmse:385732.60096
[11]	validation_0-rmse:382896.58555
[12]	validation_0-rmse:381749.42647
[13]	validation_0-rmse:379709.58522
[14]	validation_0-rmse:377954.21082
[15]	validation_0-rmse:378016.61907
[16]	validation_0-rmse:376364.60665
[17]	validation_0-rmse:375963.44145
[18]	validation_0-rmse:375674.31763
[19]	validation_0-rmse:375083.77748
[20]	validation_0-rmse:374593.30015
[21]	validation_0-rmse:372390.99883
[22]	validation_0-rmse:371509.05685
[23]	validation_0-rmse:370833.68153
[24]	validation_0-rmse:370593.03721
[25]	validation_0-rmse:370313.79977
[26]	validation_0-rmse:370002.49249
[27]	validation_0-rmse:369494.55681
[2

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

### `early_stopping_rounds`

`early_stopping_rounds` ofrece una forma de encontrar automáticamente el valor ideal para `n_estimators`. El early stopping hace que el modelo detenga las iteraciones cuando la puntuación de validación deja de mejorar, incluso si aún no hemos alcanzado el límite establecido por `n_estimators`. Es una buena práctica establecer un valor alto para `n_estimators` y luego usar `early_stopping_rounds` para encontrar el momento óptimo para detener la iteración.

Como el azar puede causar ocasionalmente una sola iteración sin mejora en la puntuación de validación, necesitas especificar cuántas rondas consecutivas de deterioro se permitirán antes de detener el entrenamiento. Establecer `early_stopping_rounds=5` es una elección razonable. En este caso, se detiene después de 5 rondas seguidas de puntuaciones de validación en descenso.

Al usar `early_stopping_rounds`, también necesitas reservar algunos datos para calcular las puntuaciones de validación; esto se hace estableciendo el parámetro `eval_set`.

Podemos modificar el ejemplo anterior para incluir el early stopping:


In [41]:
# Crear el modelo con 500 estimadores y parada anticipada
my_model = XGBRegressor(n_estimators=500,
                        early_stopping_rounds=5)  # Detener si no hay mejora después de 5 rondas

# Entrenar el modelo utilizando un conjunto de validación para early stopping
my_model.fit(X_train, y_train,
             eval_set=[(X_valid, y_valid)],  # Conjunto de validación para evaluar la mejora
             verbose=True)                  # No mostrar la salida del entrenamiento

[0]	validation_0-rmse:535046.34013
[1]	validation_0-rmse:477517.68097
[2]	validation_0-rmse:445198.04336
[3]	validation_0-rmse:425102.65133
[4]	validation_0-rmse:411945.25967
[5]	validation_0-rmse:405890.44880
[6]	validation_0-rmse:400547.88224
[7]	validation_0-rmse:396933.52512
[8]	validation_0-rmse:393692.87844
[9]	validation_0-rmse:388803.91233
[10]	validation_0-rmse:385732.60096
[11]	validation_0-rmse:382896.58555
[12]	validation_0-rmse:381749.42647
[13]	validation_0-rmse:379709.58522
[14]	validation_0-rmse:377954.21082
[15]	validation_0-rmse:378016.61907
[16]	validation_0-rmse:376364.60665
[17]	validation_0-rmse:375963.44145
[18]	validation_0-rmse:375674.31763
[19]	validation_0-rmse:375083.77748
[20]	validation_0-rmse:374593.30015
[21]	validation_0-rmse:372390.99883
[22]	validation_0-rmse:371509.05685
[23]	validation_0-rmse:370833.68153
[24]	validation_0-rmse:370593.03721
[25]	validation_0-rmse:370313.79977
[26]	validation_0-rmse:370002.49249
[27]	validation_0-rmse:369494.55681
[2

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Si más adelante deseas ajustar un modelo con todos tus datos, establece `n_estimators` en el valor que encontraste como óptimo al usar parada anticipada (early stopping).

### `learning_rate`

En lugar de obtener predicciones simplemente sumando las predicciones de cada modelo componente, podemos multiplicar las predicciones de cada modelo por un número pequeño (conocido como **tasa de aprendizaje** o *learning rate*) antes de sumarlas.

Esto significa que cada árbol que añadimos al ensamblado contribuye menos. Por lo tanto, podemos establecer un valor más alto para `n_estimators` sin sobreajustar el modelo. Si usamos parada anticipada, el número apropiado de árboles se determinará automáticamente.

En general, una tasa de aprendizaje pequeña junto con un gran número de estimadores producirá modelos de XGBoost más precisos, aunque también tomará más tiempo entrenarlos, ya que se requieren más iteraciones del ciclo. Por defecto, XGBoost establece `learning_rate=0.1`.

Si Modificamos el ejemplo anterior para cambiar la tasa de aprendizaje produce el siguiente código:


In [48]:
# Crear el modelo con una tasa de aprendizaje reducida y mayor número de estimadores
my_model = XGBRegressor(n_estimators=5000,          # Incrementamos el numero de estimadores
                        early_stopping_rounds=5,    # Detener si no hay mejora después de 5 rondas
                        learning_rate=0.05)         # Ajustamos la tasa de aprendizaje en 0.05

# Entrenar el modelo con parada anticipada y conjunto de validación
my_model.fit(X_train, y_train,
             eval_set=[(X_valid, y_valid)],  # Evaluar en el conjunto de validación
             verbose=True)                  # No mostrar mensajes durante el entrenamiento

[0]	validation_0-rmse:610600.88216
[1]	validation_0-rmse:595228.83114
[2]	validation_0-rmse:581004.97047
[3]	validation_0-rmse:567803.06131
[4]	validation_0-rmse:555815.46952
[5]	validation_0-rmse:544347.64857
[6]	validation_0-rmse:534024.08015
[7]	validation_0-rmse:523810.84026
[8]	validation_0-rmse:514587.57320
[9]	validation_0-rmse:506305.04891
[10]	validation_0-rmse:498666.96177
[11]	validation_0-rmse:490835.83185
[12]	validation_0-rmse:483846.03508
[13]	validation_0-rmse:477815.80825
[14]	validation_0-rmse:471516.73795
[15]	validation_0-rmse:466075.03817
[16]	validation_0-rmse:461158.75994
[17]	validation_0-rmse:456163.01091
[18]	validation_0-rmse:451610.93800
[19]	validation_0-rmse:447469.83449
[20]	validation_0-rmse:443733.87030
[21]	validation_0-rmse:440450.20707
[22]	validation_0-rmse:437427.93341
[23]	validation_0-rmse:434303.66595
[24]	validation_0-rmse:431591.19825
[25]	validation_0-rmse:428404.31463
[26]	validation_0-rmse:425647.45119
[27]	validation_0-rmse:422785.90823
[2

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=5000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

### `n_jobs`

En conjuntos de datos grandes, donde el tiempo de ejecución es una consideración importante, puedes usar paralelismo para construir tus modelos más rápidamente. Es común establecer el parámetro `n_jobs` igual al número de núcleos de tu máquina. En conjuntos de datos pequeños, esto no será de ayuda.

El modelo resultante no será mejor, por lo que microoptimizar el tiempo de ajuste suele ser solo una distracción. Sin embargo, es útil en conjuntos de datos grandes donde de otro modo pasarías mucho tiempo esperando durante la ejecución del comando `fit`.

Aquí tienes el ejemplo modificado:


In [28]:
# Crear el modelo con paralelismo (4 núcleos), tasa de aprendizaje reducida y early stopping
my_model = XGBRegressor(n_estimators=1000,
                        early_stopping_rounds=5,
                        learning_rate=0.05,
                        n_jobs=4)

# Entrenar el modelo utilizando el conjunto de validación para aplicar early stopping
my_model.fit(X_train, y_train,
             eval_set=[(X_valid, y_valid)],  # Evaluación del rendimiento durante el entrenamiento
             verbose=False)                  # No mostrar salida detallada


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=4,
             num_parallel_tree=None, random_state=None, ...)

# Conclusión

[XGBoost](https://xgboost.readthedocs.io/en/latest/) es una de las bibliotecas de software líderes para trabajar con datos tabulares estándar (el tipo de datos que se almacenan en DataFrames de Pandas, a diferencia de tipos de datos más complejos como imágenes o videos). Con un ajuste cuidadoso de los parámetros, puedes entrenar modelos altamente precisos.